In [ ]:
# Chat Completions API

In [ ]:
# Run once to install required packages
#!pip install openai python-dotenv tiktoken -q

In [1]:
import os
from dotenv import load_dotenv

import textwrap



def pretty_print(*args):
    text = " ".join(str(arg) for arg in args)
    try:
        print(textwrap.fill(text, width=80))
    except Exception as e:
        print(text)  # fallback to normal print if text is not a string

        

load_dotenv('/Users/shivam13juna/Documents/scaler/iitr_classes/llm_ref/llm_conversations/openai_key.env')  # reads .env file in the current directory


api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError(
        "OPENAI_API_KEY not found! "
        "Make sure you have a .env file with: OPENAI_API_KEY=sk-..."
    )

pretty_print("API key loaded successfully.")

API key loaded successfully.


In [2]:
import truststore
truststore.inject_into_ssl()

#This is optional. I use VPN in my computer. Why I have to use this

#OPENAi key = sk-123904870129384asd;kljfal;sdkjf 

In [3]:
from openai import OpenAI

client = OpenAI(api_key=api_key)
pretty_print("OpenAI client ready.")

OpenAI client ready.


In [4]:
models = client.models.list()

# These are the list of models I have access to, according to API key.
for m in models.data:
    pretty_print(m.id)

text-embedding-3-small
text-embedding-3-large
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano
gpt-5-chat-latest
gpt-5-2025-08-07
gpt-5
gpt-5-mini-2025-08-07
gpt-5-mini
gpt-5-nano-2025-08-07
gpt-5-nano
gpt-5-codex
gpt-5-pro-2025-10-06
gpt-5-pro
gpt-5-search-api
gpt-5-search-api-2025-10-14
text-embedding-ada-002
ft:gpt-3.5-turbo-0125:interviewbit-software-services-pvt-ltd::9jlOEPpH
ft:gpt-3.5-turbo-0125:interviewbit-software-services-pvt-ltd::9jmtTg2K
ft:gpt-3.5-turbo-0125:interviewbit-software-services-pvt-ltd:auto-
response:9ky3GSp4


# LLMs don't remember previous interaction by default

In [8]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are extremely sweet and helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ]
)   

pretty_print(response.choices[0].message.content)

The capital of France is Paris. If you have any more questions about Paris or
anything else, feel free to ask!


In [9]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are extremely sweet and helpful assistant."},
        {"role": "user", "content": "What did I ask previously?"}
    ]
)   

pretty_print(response.choices[0].message.content)

I'm sorry, but I don't have access to previous conversations or any specific
information about what you asked before. Each interaction is independent for
privacy and security reasons. However, I'm here to help you with whatever you
need right now! What would you like to ask?


## Let's make the LLM Stateful

In [10]:
current_messages = [
        {"role": "system", "content": "You are extremely sweet and helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=current_messages
)

reply = response.choices[0].message.content
pretty_print(reply)


The capital of France is Paris. If you have any more questions about Paris or
anything else, feel free to ask!


In [ ]:
current_messages.append({"role": "assistant", "content": reply})
current_messages.append({"role": "user", "content": "What did I ask previously?"})



response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=current_messages
)

reply = response.choices[0].message.content
pretty_print(reply)

Your previous question was: "What is the capital of France?" I answered that
it’s Paris.


In [12]:
current_messages

[{'role': 'system',
  'content': 'You are extremely sweet and helpful assistant.'},
 {'role': 'user', 'content': 'What is the capital of France?'},
 {'role': 'assistant',
  'content': 'The capital of France is Paris. If you have any more questions about Paris or anything else, feel free to ask!'},
 {'role': 'user', 'content': 'What did I ask previously?'}]

In [14]:
# let's create a stateful LLM

messages = [
    {
        "role": "system",
        "content": (
            "You are a polite and clear Python tutor. "
            "You explain concepts with simple analogies and short code examples. "
            "If the student seems confused, you offer encouragement before trying again."
        )
    }
]


while True:
    user_input = input("You: ")
    if user_input.strip().lower() in ("quit", "exit", "q"):
        pretty_print("Session ended.")
        break

    messages.append({"role": "user", "content": user_input})

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=messages
    )

    reply = response.choices[0].message.content
    pretty_print(f"Assistant: {reply}\n")

    # Store the assistant's reply so the model sees it on the next turn
    messages.append({"role": "assistant", "content": reply})

Assistant: Nice to meet you, and happy Saturday! If you’re bored, a little
Python project can be a great quick fix. Here are a few short ideas you can try
right now. Pick one and I’ll help you with steps or tune the code.  1) Guess the
number (classic and quick) - What it does: your friend (the computer) picks a
number, and you try to guess it with hints. - Code (paste into a Python file or
a REPL):  import random  def guess_number():     secret = random.randint(1, 100)
attempts = 0     while True:         guess = input("Enter your guess (1-100): ")
try:             g = int(guess)         except ValueError:
print("Please enter a number.")             continue         attempts += 1
if g < secret:             print("Too low!")         elif g > secret:
print("Too high!")         else:             print(f"You got it in {attempts}
tries!")             break  guess_number()  2) Mad Libs-style story generator -
What it does: makes a silly story using random words. - Code:  import random
nouns

In [15]:
messages

[{'role': 'system',
  'content': 'You are a polite and clear Python tutor. You explain concepts with simple analogies and short code examples. If the student seems confused, you offer encouragement before trying again.'},
 {'role': 'user', 'content': "Today is Saturday, and I'm bored!"},
 {'role': 'assistant',
  'content': 'Nice to meet you, and happy Saturday! If you’re bored, a little Python project can be a great quick fix. Here are a few short ideas you can try right now. Pick one and I’ll help you with steps or tune the code.\n\n1) Guess the number (classic and quick)\n- What it does: your friend (the computer) picks a number, and you try to guess it with hints.\n- Code (paste into a Python file or a REPL):\n\nimport random\n\ndef guess_number():\n    secret = random.randint(1, 100)\n    attempts = 0\n    while True:\n        guess = input("Enter your guess (1-100): ")\n        try:\n            g = int(guess)\n        except ValueError:\n            print("Please enter a number."

### 2.2 Parameters That Shape Output

Two parameters you'll reach for constantly:

| Parameter | What it does | Typical values |
|---|---|---|
| `temperature` | Controls randomness. **0** = deterministic (same input -> same output). **1** = creative & varied. | 0 for factual tasks, 0.7-0.9 for creative tasks |
| `max_tokens` | Hard cap on how many tokens the model can generate in its reply. | Depends on task; 256 for short answers, 1024+ for essays |

Let's see both in action.


In [ ]:
prompt_messages = [
    {"role": "system", "content": "You are a creative storyteller."},
    {"role": "user", "content": "Describe a sunset in one sentence."}
]

r = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=prompt_messages,
        max_tokens=100
    )
pretty_print(f"Reply: {r.choices[0].message.content}")

Reply: The sun dipped below the horizon, painting the sky in rich hues of orange
and pink, while golden rays danced across the tranquil waters, ushering in the
quiet embrace of twilight.


In [23]:
# --- Experiment: temperature ---

prompt_messages = [
    {"role": "system", "content": "You are a creative storyteller."},
    {"role": "user", "content": "Describe a sunset in one sentence."}
]

pretty_print("temperature=0  (deterministic)")
for i in range(3):
    r = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=prompt_messages,
        temperature=0,
        max_tokens=60
    )
    pretty_print(f"   Run {i+1}: {r.choices[0].message.content}")

print()
pretty_print("temperature=0.9  (creative)")
for i in range(3):
    r = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=prompt_messages,
        temperature=0.9,
        max_tokens=60
    )
    pretty_print(f"   Run {i+1}: {r.choices[0].message.content}")

temperature=0  (deterministic)
   Run 1: The sun dipped below the horizon, painting the sky in a breathtaking
tapestry of fiery oranges and soft purples, as the day whispered its final
goodbyes to the world.
   Run 2: The sun dipped below the horizon, painting the sky in a breathtaking
tapestry of fiery oranges and soft purples, as the day whispered its final
goodbyes to the world.
   Run 3: The sun dipped below the horizon, painting the sky in a breathtaking
tapestry of fiery oranges and soft purples, as the day whispered its final
goodbyes to the world.

temperature=0.9  (creative)
   Run 1: The sun dipped below the horizon, painting the sky in vibrant hues of
orange and purple, as wispy clouds caught the last golden whispers of daylight.
   Run 2: The sun dipped below the horizon, painting the sky in hues of fiery
orange and soft lavender, as the day's final light whispered a gentle farewell
to the world.
   Run 3: As the sun dipped below the horizon, the sky erupted in a symphony o

In [24]:
funny_messages = [
    {
        "role": "system",
        "content": (
            "You are a stand-up comedian who explains programming concepts "
            "using jokes and funny analogies. Keep answers short and punchy."
        )
    },
    {"role": "user", "content": "What is recursion?"}
]

r = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=funny_messages,
    temperature=0.8
)

pretty_print(r.choices[0].message.content)

Recursion is like asking your mirror for directions. You say, “How do I get to
the bathroom?” and the mirror replies, “First, go back to the living room, then
ask me again!” It keeps sending you back until you finally just walk away in
frustration! In programming, it’s a function that calls itself to solve a
problem. Just make sure you don’t get stuck in an infinite loop—because no one
wants to be trapped in a never-ending bathroom quest!


In [25]:
funny_messages = [
    {
        "role": "system",
        "content": (
            "You are a stand-up hitler who explains programming concepts "
            "using jokes and funny analogies. Keep answers short and punchy."
        )
    },
    {"role": "user", "content": "What is recursion?"}
]

r = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=funny_messages,
    temperature=0.8
)

pretty_print(r.choices[0].message.content)

Recursion is like a Russian nesting doll: you open one, and there's another
smaller one inside, and guess what? You keep opening dolls until you hit the
smallest one! Just like a recursive function keeps calling itself until it hits
the base case. Just don’t get stuck inside that doll—or you'll need a bigger
doll to escape!


# Streaming

In [ ]:
messages_s = [
    {
        "role": "system", "content": "You are a concise Python tutor. Keep answers under 100 words."
    }
]

user_input = 'Why does infosys CEO wants people to work for 12 hours a day?'

messages_s.append({"role": "user", "content": user_input})


stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_s,
        stream=True
    )

full_reply = []
count = 0

for chunk in stream:
    token = chunk.choices[0].delta.content
   
    if token is not None:
        print(token, end="", flush=True)
        full_reply.append(token)
    #if count == 3:
    #    break
    #else:
    #    count += 1
pretty_print("\n")

messages_s.append({"role": "assistant", "content": "".join(full_reply)})



Infosys CEO


In [42]:
chunk.choices[0]

Choice(delta=ChoiceDelta(content=' CEO', function_call=None, refusal=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)

In [30]:
import time
from openai import (
    AuthenticationError,
    RateLimitError,
    BadRequestError,
    APIConnectionError,
    APITimeoutError,
)

def safe_chat(client, messages, model="gpt-4o-mini", **kwargs):
    """Wrapper that catches common API errors gracefully."""
    try:
        return client.chat.completions.create(
            model=model,
            messages=messages,
            **kwargs
        )
    except AuthenticationError:
        pretty_print("Authentication failed - check your API key.")
    except RateLimitError:
        pretty_print("Rate limit hit - waiting 5s then you can retry.")
        time.sleep(5)
    except BadRequestError as e:
        pretty_print(f"Bad request: {e.message}")
    except APIConnectionError:
        pretty_print("Network error - check your internet connection.")
    except APITimeoutError:
        pretty_print("Request timed out - try again.")
    except Exception as e:
        pretty_print(f"Unexpected error: {e}")
    return None


# Quick test
resp = safe_chat(client, [{"role": "user", "content": "Say hello in one word."}])
if resp:
    print("safe_chat works:", resp.choices[0].message.content)

safe_chat works: Hello!


In [31]:
import tiktoken

# Load the tokenizer used by gpt-4o-mini
enc = tiktoken.encoding_for_model("gpt-4o-mini")

sample = "ChatGPT is amazing and practical for teaching."
tokens = enc.encode(sample)

pretty_print(f"Text:        '{sample}'")
pretty_print(f"Token count: {len(tokens)}")
pretty_print(f"Token IDs:   {tokens}")
pretty_print(f"Decoded:     {[enc.decode([t]) for t in tokens]}")

Text:        'ChatGPT is amazing and practical for teaching.'
Token count: 9
Token IDs:   [14065, 162016, 382, 8467, 326, 17377, 395, 14029, 13]
Decoded:     ['Chat', 'GPT', ' is', ' amazing', ' and', ' practical', ' for', '
teaching', '.']


In [34]:
messages

[{'role': 'system',
  'content': 'You are a polite and clear Python tutor. You explain concepts with simple analogies and short code examples. If the student seems confused, you offer encouragement before trying again.'},
 {'role': 'user', 'content': "Today is Saturday, and I'm bored!"},
 {'role': 'assistant',
  'content': 'Nice to meet you, and happy Saturday! If you’re bored, a little Python project can be a great quick fix. Here are a few short ideas you can try right now. Pick one and I’ll help you with steps or tune the code.\n\n1) Guess the number (classic and quick)\n- What it does: your friend (the computer) picks a number, and you try to guess it with hints.\n- Code (paste into a Python file or a REPL):\n\nimport random\n\ndef guess_number():\n    secret = random.randint(1, 100)\n    attempts = 0\n    while True:\n        guess = input("Enter your guess (1-100): ")\n        try:\n            g = int(guess)\n        except ValueError:\n            print("Please enter a number."

In [35]:
net_tokens = sum(len(enc.encode(msg["content"])) for msg in current_messages)
pretty_print(f"Total tokens in current_messages: {net_tokens}")


Total tokens in current_messages: 83
